In [ ]:
!open .

In [8]:
# Setting Up Imports 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager



import copy

from urllib.parse import urlparse
import numpy as np
import pandas as pd
import re 
from pathlib import Path
import os
import time
from datetime import date
from datetime import timedelta
from datetime import datetime
import dateutil.parser as parser

today = date.today()
cwd = os.getcwd()


# Setting up browser environment

userprofile = 'samarth'
options = webdriver.ChromeOptions()
options.add_argument(f"user-data-dir={cwd}/."+userprofile+"UserProfile")
driver = webdriver.Chrome(ChromeDriverManager().install() , options=options)
driver.get("https://web.whatsapp.com/")


def F_SEARCH_2(driverVar,contact_number):

    driverVar.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.CONTROL + Keys.COMMAND + '//')
    time.sleep(1)


    FORM_HEADER_CLASS = '_1Jn3C' # Label Class
    FORM_CLASS = '_13NKt copyable-text selectable-text'

    formHeadingClassList = driverVar.find_elements(By.XPATH,f"//label[@class='{FORM_HEADER_CLASS}']")

    print(len(formHeadingClassList))


    if len(formHeadingClassList)==1:
        formHeadingClass = formHeadingClassList[0]
        formElementsList = formHeadingClass.find_elements(By.XPATH,f".//div[@class='{FORM_CLASS}']")

        print(len(formElementsList))

        if len(formElementsList)>0:
            formElement = formElementsList[0]
            formElement.click();
            formElement.clear();
            formElement.send_keys(f'{contact_number}')
            time.sleep(4)
            formElement.send_keys(Keys.RETURN)   
            
            
def F_SINGLE_SEND(driverVar,text_message):
    CHAT_HEADER_CLASS = '_1UWac _1LbR4'
    ALT_CHAT_HEADER_CLASS = '_1UWac _1LbR4 focused'
    CHAT_CLASS = '_13NKt copyable-text selectable-text'

    print('Trying Unfocussed')
    chatHeadingClassList = driverVar.find_elements(By.XPATH,f"//div[@class='{CHAT_HEADER_CLASS}']")
    if len(chatHeadingClassList)==0:
        print('Unfocussed failed, trying focussed;')
        chatHeadingClassList = driverVar.find_elements(By.XPATH,f"//div[@class='{ALT_CHAT_HEADER_CLASS}']")
    print(f'Length of Chat Header Div => {len(chatHeadingClassList)}')

    if len(chatHeadingClassList)==1:
        chatHeadingClass = chatHeadingClassList[0]
        chatElementsList = chatHeadingClass.find_elements(By.XPATH,f".//div[@class='{CHAT_CLASS}']")
        print(len(chatElementsList))
        if len(chatElementsList)>0:
            chatElement = chatElementsList[0]
            chatElement.click();
            time.sleep(1)
            chatElement.clear();
            chatElement.send_keys(f'{text_message}')
            time.sleep(2)
            chatElement.send_keys(Keys.RETURN)




def F_DOCUMENT_SEND_NEW(driverVar,op,absolute_path):


    if op == 'document':
        operation = 'attach-document'
    if op == 'image':
        operation = 'attach-image'

    print(f'Your Selected Operation is {operation}')

    BUTTON_CLASS = '_26lC3'
    ATTACH_BUTTON_INNER_CLASS = '_1HnQz'
    AttachButtonElement = -1 
    SEND_BUTTON_CLASS = '_165_h _2HL9j'




    print('[1] D__Search Page Buttons__: Scraping Button List ')
    ButtonClassList = driverVar.find_elements(By.XPATH,f"//div[@class='{BUTTON_CLASS}']")
    if len(ButtonClassList)<0 :
        print('[1] D__Search Page Buttons__: ERROR : Button List Not Found ')

    time.sleep(2)
    print('button class list',ButtonClassList)
    for recepient in ButtonClassList:
        try:
            if recepient.get_attribute('title') == 'Attach':
                AttachButtonElement = recepient
                print(AttachButtonElement)
                print('[2] D__Filter Page Buttons__: SUCCESS : Button Found')
        except Exception:
            print('[2] D__Filter Page Buttons__: ERROR : Problem in reading button attribute, skipping...')


    AttachButtonElement.click()   
    time.sleep(1)




    print('[3] D__Search Attach Inner Buttons__: Scraping Button List ')
    ButtonClassList = driverVar.find_elements(By.XPATH,f"//ul[@class='{ATTACH_BUTTON_INNER_CLASS}']")
    if len(ButtonClassList)==0 :
        print('[3] D__Search Attach Inner Buttons__: ERROR : Button List Not Found : Aborting')
        return('0')
    else:
        print('Inner Button Listing Found')






    DataEntryButtons = ButtonClassList[0].find_elements(By.XPATH,f".//li")
    print(len(DataEntryButtons))

    for recepients in DataEntryButtons:
#         try:
        ButtonIdentifier = recepients.find_elements(By.XPATH,f".//span")
        
        identified = ButtonIdentifier[0].get_attribute('data-testid')
        print(f"Identified Button : {identified}")


        InputElement = recepients.find_elements(By.XPATH,f".//input")
        print('d',InputElement)

        if (identified == operation):
            time.sleep(2)
            InputElement[0].send_keys(absolute_path)
            break
            time.sleep(2)
                
#         except Exception:
#             print('Button Identifier Empty, skipping')
            

            
    time.sleep(2)
    SendButtonList = driverVar.find_elements(By.XPATH,f"//div[@class='{SEND_BUTTON_CLASS}']")
    time.sleep(1)
    SendButtonList[0].click()
    time.sleep(1)
    




           
            
def GET_MESSAGE_LIST(tagCollection,db,tagColumns):
    
    MESSAGE_LIST = []
    
    if len(tagCollection)==0:
        print('No Tags available')
    elif len(tagCollection)==1:
        print('\n--------Case 1 - SEARCHING FOR 1 TAG IN USER')
        for index, row in db.iterrows():
            print(f'Iterating over user <<{row["First Name"]} {row["Last Name"]}>> to find tags...')
            taglist = []
            for tag in tagColumns:
                if type(row[tag]) == str:
                    taglist.append(row[tag])
            taglist.sort()
            print(taglist)

            if tagCollection[0] in taglist:
                print('         Found Match, Added to Message List\n')
                MESSAGE_LIST.append({
                    'fn':row['First Name'],
                    'ln':row['Last Name'],
                    'contact':row['Mobile Number'],
                })


    elif len(tagCollection)>1:
        print('\n--------Case 2 - SEARCHING FOR MULTIPLE TAGS IN USER')
        for index, row in db.iterrows():
            print(f'Iterating over user <<{row["First Name"]} {row["Last Name"]}>> to find tags...')
            taglist = []
            for tag in tagColumns:
                if type(row[tag]) == str:
                    taglist.append(row[tag])
            taglist.sort()
            print(taglist)
            if tagCollection == taglist:
                print('           Found Match, Added to Message List')
                MESSAGE_LIST.append({
                    'fn':row['First Name'],
                    'ln':row['Last Name'],
                    'contact':row['Mobile Number'],
                })
    return MESSAGE_LIST
            
# Reading Database and Getting Tag Options 

db = pd.read_csv('./database.csv')


tagColumns = db.columns[4:]
tagCollection = []

for tagColumnIndex in tagColumns:
    tags = db[tagColumnIndex].unique().tolist()


    tagCollection = tagCollection + tags
    
# Variable containing list of available tags 
tagCollectionFiltered = [tag for tag in tagCollection if type(tag) != float ]






[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/flipflops/.wdm/drivers/chromedriver/mac64/95.0.4638.17/chromedriver] found in cache
/usr/local/Caskroom/miniconda/base/envs/tpc-wa/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


In [5]:
k  = Path ('./RVintro.pdf')

In [9]:
F_DOCUMENT_SEND_NEW(driver,'document',str(k.resolve()))

Your Selected Operation is attach-document
[1] D__Search Page Buttons__: Scraping Button List 
button class list [<selenium.webdriver.remote.webelement.WebElement (session="a502c3d8bd2ec5ac57dc075f0d0008c4", element="40777435-7d88-4a65-9bde-0609e5ab4a7e")>, <selenium.webdriver.remote.webelement.WebElement (session="a502c3d8bd2ec5ac57dc075f0d0008c4", element="42e3fc53-6741-4834-b9fc-44bd15a9d079")>, <selenium.webdriver.remote.webelement.WebElement (session="a502c3d8bd2ec5ac57dc075f0d0008c4", element="fbdb3879-1831-4b51-b55d-7308e84638fd")>, <selenium.webdriver.remote.webelement.WebElement (session="a502c3d8bd2ec5ac57dc075f0d0008c4", element="e112edf9-2406-447f-bd84-74eb5ead2294")>, <selenium.webdriver.remote.webelement.WebElement (session="a502c3d8bd2ec5ac57dc075f0d0008c4", element="adbb1123-6c6a-48b5-93b2-111c3c09b313")>, <selenium.webdriver.remote.webelement.WebElement (session="a502c3d8bd2ec5ac57dc075f0d0008c4", element="e8ed47d2-7b58-4195-92d9-ee04b331079a")>]
<selenium.webdriver.re

In [3]:
!open .